In [ ]:
# conda env create --force -f environment.yml

In [ ]:
import warnings

from typing import List

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import sklearn
import scipy
import seaborn as sns

from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from tqdm import tqdm

In [ ]:
# Filter: RuntimeWarning: invalid value encountered in divide
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in divide")

In [ ]:
plt.style.use("seaborn-v0_8-deep")

font = {"family": "serif", "size": 14}

matplotlib.rc("font", **font)

# Load data

In [ ]:
median_ensemble_test = pd.read_csv("./delivery_1_cv_v7_seeds/split_v7_2021_test_cc_dispatcher_precision_recall_harmonic_mean_median_ensemble_individual_predictions.csv")
median_ensemble_val = pd.read_csv("./delivery_1_cv_v7_seeds/split_v7_2021_test_cc_dispatcher_precision_recall_harmonic_mean_median_ensemble_individual_predictions_val.csv")

In [ ]:
all_ensembles_test = pd.read_csv("./delivery_1_cv_v7_seeds/split_v7_2021_test_cc_dispatcher_precision_recall_harmonic_mean_predictions.csv")
all_ensembles_val = pd.read_csv("./delivery_1_cv_v7_seeds/split_v7_2021_test_cc_dispatcher_precision_recall_harmonic_mean_predictions_val.csv")

In [ ]:
column_map = {
    "logits 20": "logits 1",
    "logits 21": "logits 2",
    "logits 22": "logits 3",
    "logits 23": "logits 4",
    "logits 24": "logits 5",
    "probs 20": "probs 1",
    "probs 21": "probs 2",
    "probs 22": "probs 3",
    "probs 23": "probs 4",
    "probs 24": "probs 5",
}
median_ensemble_test.rename(columns=column_map, inplace=True)
median_ensemble_test["ensemble_preds"] = median_ensemble_test["ensemble_probs"] > 0.5

median_ensemble_val.rename(columns=column_map, inplace=True)
median_ensemble_val["ensemble_preds"] = median_ensemble_val["ensemble_probs"] > 0.5

median_ensemble_test

In [ ]:
from sklearn.utils import (column_or_1d)
from sklearn.utils.validation import check_consistent_length, _check_pos_label_consistency


In [ ]:
def calibration_curve(
    y_true,
    y_prob,
    *,
    pos_label=None,
    n_bins=5,
    strategy="uniform",
):
    """Compute true and predicted probabilities for a calibration curve.

    The method assumes the inputs come from a binary classifier, and
    discretize the [0, 1] interval into bins.

    Calibration curves may also be referred to as reliability diagrams.

    Read more in the :ref:`User Guide <calibration>`.

    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        True targets.

    y_prob : array-like of shape (n_samples,)
        Probabilities of the positive class.

    pos_label : int, float, bool or str, default=None
        The label of the positive class.

        .. versionadded:: 1.1

    n_bins : int, default=5
        Number of bins to discretize the [0, 1] interval. A bigger number
        requires more data. Bins with no samples (i.e. without
        corresponding values in `y_prob`) will not be returned, thus the
        returned arrays may have less than `n_bins` values.

    strategy : {'uniform', 'quantile'}, default='uniform'
        Strategy used to define the widths of the bins.

        uniform
            The bins have identical widths.
        quantile
            The bins have the same number of samples and depend on `y_prob`.

    Returns
    -------
    prob_true : ndarray of shape (n_bins,) or smaller
        The proportion of samples whose class is the positive class, in each
        bin (fraction of positives).

    prob_pred : ndarray of shape (n_bins,) or smaller
        The mean predicted probability in each bin.

    References
    ----------
    Alexandru Niculescu-Mizil and Rich Caruana (2005) Predicting Good
    Probabilities With Supervised Learning, in Proceedings of the 22nd
    International Conference on Machine Learning (ICML).
    See section 4 (Qualitative Analysis of Predictions).

    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.calibration import calibration_curve
    >>> y_true = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1])
    >>> y_pred = np.array([0.1, 0.2, 0.3, 0.4, 0.65, 0.7, 0.8, 0.9,  1.])
    >>> prob_true, prob_pred = calibration_curve(y_true, y_pred, n_bins=3)
    >>> prob_true
    array([0. , 0.5, 1. ])
    >>> prob_pred
    array([0.2  , 0.525, 0.85 ])
    """
    y_true = column_or_1d(y_true)
    y_prob = column_or_1d(y_prob)
    check_consistent_length(y_true, y_prob)
    pos_label = _check_pos_label_consistency(pos_label, y_true)

    if y_prob.min() < 0 or y_prob.max() > 1:
        raise ValueError("y_prob has values outside [0, 1].")

    labels = np.unique(y_true)
    if len(labels) > 2:
        raise ValueError(
            f"Only binary classification is supported. Provided labels {labels}."
        )
    y_true = y_true == pos_label

    if isinstance(n_bins, (list, np.ndarray)):
        bins = np.asarray(n_bins)
        if (bins < 0).any() or (bins > 1).any():
            raise ValueError("n_bins must be between 0 and 1")
        if (bins[:-1] >= bins[1:]).any():
            raise ValueError("bins must increase monotonically")
    elif strategy == "quantile":  # Determine bin edges by distribution of data
        quantiles = np.linspace(0, 1, n_bins + 1)
        bins = np.percentile(y_prob, quantiles * 100)
    elif strategy == "uniform":
        bins = np.linspace(0.0, 1.0, n_bins + 1)
    else:
        raise ValueError(
            "Invalid entry to 'strategy' input. Strategy "
            "must be either 'quantile' or 'uniform'."
        )

    binids = np.searchsorted(bins[1:-1], y_prob)

    bin_sums = np.bincount(binids, weights=y_prob, minlength=len(bins))
    bin_true = np.bincount(binids, weights=y_true, minlength=len(bins))
    bin_total = np.bincount(binids, minlength=len(bins))

    # nonzero = bin_total != 0
    # prob_true = bin_true[nonzero] / bin_total[nonzero]
    # prob_pred = bin_sums[nonzero] / bin_total[nonzero]
    
    prob_true = bin_true / bin_total
    prob_pred = bin_sums / bin_total

    return prob_true, prob_pred



In [ ]:
def plot_histogram(arrays: List[np.ndarray], labels: List, **kwargs):
    
    density = kwargs.get("density", False)
    fig, ax = plt.subplots(figsize=(6.4, 4.8))

    for array, label in zip(arrays, labels):
        ax.hist(array, bins=kwargs.get("bins", 50), alpha=kwargs.get("alpha", 0.5), label=label, density=density)

    ax.set_yscale(kwargs.get("yscale", "log"))
    ax.set_xlabel("Predicted probability")
    if density:
        ax.set_ylabel("Density")
    else:
        ax.set_ylabel("Count")
    ax.legend()
    return fig, ax

In [ ]:
def plot_kde_1d(arrays: List[np.ndarray], labels: List, **kwargs):
    fig, ax = plt.subplots(figsize=(6.4, 4.8))

    for array, label in zip(arrays, labels):
        ax = sns.kdeplot(array, ax=ax, label=label, bw_method=kwargs.get("bw_method", 0.1))

    ax.set_xlabel(kwargs.get("xlabel", "Predicted probability"))
    ax.set_ylabel(kwargs.get("ylabel", "Log-density"))
    ax.legend()
    ax.set_yscale(kwargs.get("yscale", "log"))
    ax.set_xlim(kwargs.get("xlim", (0, 1)))
    return fig, ax

In [ ]:
def plot_calibration_curve(targets: List[np.ndarray], model_probs: List[np.ndarray], labels=None, ax=None, **kwargs):
    if not isinstance(targets, list):
        targets = [targets]
    if not isinstance(model_probs, list):
        model_probs = [model_probs]

    if len(model_probs) != len(targets):
        assert len(model_probs) == 1 or len(targets) == 1, "Number of models and targets must be equal or 1"
        if len(model_probs) == 1:
            model_probs = model_probs * len(targets)
        else:
            targets = targets * len(model_probs)

    if ax is None:
        fig, ax = plt.subplots(figsize=(6.4, 4.8))
        ax.plot([0, 1], [0, 1], "k:", label="Perfect calibration")
    else:
        fig = None

    for target, probs, label in zip(targets, model_probs, labels):
        prob_true, prob_pred = calibration_curve(
            target,
            probs,
            n_bins=kwargs.get("n_bins", 20),
            strategy=kwargs.get("strategy", "uniform"),
        )
        ax.plot(prob_pred, prob_true, marker="o", markersize=3, label=label)

    ax.set_xlabel("Mean predicted probability")
    ax.set_ylabel("Fraction of positives")
    ax.legend()
    return fig, ax

In [ ]:
fig, ax = plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=[median_ensemble_test["ensemble_probs"], median_ensemble_test["probs 1"]],
    labels=["Ensemble", "Single model"],
    n_bins=20,
)
# fig.savefig("calibration_curve_ensemble_and_single_model_uncalibrated.pdf", bbox_inches="tight")
fig, ax = plot_histogram(arrays=[median_ensemble_test["ensemble_probs"], median_ensemble_test["probs 1"]], labels=["Ensemble", "Constituent model 1"], bins=30)
fig.savefig("histogram_ensemble_and_single_model.pdf", bbox_inches="tight")


In [ ]:
fig, ax = plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=[
        median_ensemble_test["ensemble_probs"],
        median_ensemble_test["probs 1"],
        median_ensemble_test["probs 2"],
        median_ensemble_test["probs 3"],
        median_ensemble_test["probs 4"],
        median_ensemble_test["probs 5"],
    ],
    labels=["Ensemble", "Constituent model 1", "Constituent model 2", "Constituent model 3", "Constituent model 4", "Constituent model 4"],
    n_bins=20,
)
fig.savefig("calibration_curve_ensemble_and_all_models_uncalibrated.pdf", bbox_inches="tight")
plot_histogram(
    arrays=[
        median_ensemble_test["ensemble_probs"],
        median_ensemble_test["probs 1"],
        median_ensemble_test["probs 2"],
        median_ensemble_test["probs 3"],
        median_ensemble_test["probs 4"],
        median_ensemble_test["probs 5"],
    ],
    labels=["Ensemble", "Constituent model 1", "Constituent model 2", "Constituent model 3", "Constituent model 4", "Constituent model 4"],
    alpha=0.3
)

fig, ax = plot_kde_1d(
    arrays=[
        median_ensemble_test["ensemble_probs"],
        median_ensemble_test["probs 1"],
        median_ensemble_test["probs 2"],
        median_ensemble_test["probs 3"],
        median_ensemble_test["probs 4"],
        median_ensemble_test["probs 5"],
    ],
    labels=["Ensemble", "Constituent model 1", "Constituent model 2", "Constituent model 3", "Constituent model 4", "Constituent model 4"],
)
fig.savefig("kde_ensemble_and_all_models.pdf", bbox_inches="tight")

In [ ]:
is_male = median_ensemble_test["gender"] == "M"
is_female = median_ensemble_test["gender"] == "K"
plot_calibration_curve(
    targets=[median_ensemble_test["y"][is_male], median_ensemble_test["y"][is_female]],
    model_probs=[median_ensemble_test["ensemble_probs"][is_male], median_ensemble_test["ensemble_probs"][is_female]],
    labels=["Male", "Female"],
    n_bins=20,
)
plot_histogram(
    arrays=[median_ensemble_test["ensemble_probs"][is_male], median_ensemble_test["ensemble_probs"][is_female]],
    labels=["Male", "Female"],
)

In [ ]:
condition = median_ensemble_test["h"]

plot_calibration_curve(
    targets=[median_ensemble_test["y"][condition], median_ensemble_test["y"][~condition]],
    model_probs=[median_ensemble_test["ensemble_probs"][condition], median_ensemble_test["ensemble_probs"][~condition]],
    labels=["Call-taker recognition", "No call-taker recognition"],
    n_bins=20,
)
plot_histogram(
    arrays=[median_ensemble_test["ensemble_probs"][condition], median_ensemble_test["ensemble_probs"][~condition]],
    labels=["Call-taker recognition", "No call-taker recognition"],
)

In [ ]:
plot_histogram(
    arrays=[median_ensemble_test["ensemble_probs"][condition]],
    labels=["Call-taker recognition"],
    yscale="linear"
)

In [ ]:
condition = median_ensemble_test["h"] == median_ensemble_test["ensemble_preds"]

plot_calibration_curve(
    targets=[median_ensemble_test["y"][condition], median_ensemble_test["y"][~condition]],
    model_probs=[median_ensemble_test["ensemble_probs"][condition], median_ensemble_test["ensemble_probs"][~condition]],
    labels=["Model/call-taker agreement", "Model/call-taker disagreement"],
    n_bins=20,
)
plot_histogram(
    arrays=[median_ensemble_test["ensemble_probs"][condition], median_ensemble_test["ensemble_probs"][~condition]],
    labels=["Model/call-taker agreement", "Model/call-taker disagreement"],
)

In [ ]:
is_old = median_ensemble_test["age"] >= 65
plot_calibration_curve(
    targets=[median_ensemble_test["y"][is_old], median_ensemble_test["y"][~is_old]],
    model_probs=[median_ensemble_test["ensemble_probs"][is_old], median_ensemble_test["ensemble_probs"][~is_old]],
    labels=["65+", "18-65"],
    n_bins=20,
)
plot_histogram(
    arrays=[median_ensemble_test["ensemble_probs"][is_old], median_ensemble_test["ensemble_probs"][~is_old]],
    labels=["65+", "18-65"],
)

In [ ]:
condition = median_ensemble_test["y"]

plot_calibration_curve(
    targets=[median_ensemble_test["y"][condition], median_ensemble_test["y"][~condition]],
    model_probs=[median_ensemble_test["ensemble_probs"][condition], median_ensemble_test["ensemble_probs"][~condition]],
    labels=["Model/call-taker agreement", "Model/call-taker disagreement"],
    n_bins=20,
)
plot_histogram(
    arrays=[median_ensemble_test["ensemble_probs"][condition], median_ensemble_test["ensemble_probs"][~condition]],
    labels=["Model/call-taker agreement", "Model/call-taker disagreement"],
)

# Calibration

## Platt scaling

In [ ]:
logistic = LogisticRegression(penalty="none", fit_intercept=True)
logistic.fit(median_ensemble_val["ensemble_probs"].to_numpy()[:, np.newaxis], median_ensemble_val["y"].to_numpy())
ensemble_probs_logistic = logistic.predict_proba(median_ensemble_test["ensemble_probs"].to_numpy()[:, np.newaxis])[:, 1]
ensemble_probs_logistic_val = logistic.predict_proba(median_ensemble_val["ensemble_probs"].to_numpy()[:, np.newaxis])[:, 1]

individual_probs_logistic = []
individual_probs_logistic_val = []
for i in range(1, 6):
    logistic_i = LogisticRegression(penalty="none", fit_intercept=True)
    logistic_i.fit(median_ensemble_val[f"probs {i}"].to_numpy()[:, np.newaxis], median_ensemble_val["y"].to_numpy())
    individual_probs_logistic += [logistic_i.predict_proba(median_ensemble_test[f"probs {i}"].to_numpy()[:, np.newaxis])[:, 1]]
    individual_probs_logistic_val += [logistic_i.predict_proba(median_ensemble_val[f"probs {i}"].to_numpy()[:, np.newaxis])[:, 1]]

In [ ]:
logistic.predict_proba(np.array([[0.8]]))

In [ ]:
logistic.coef_, logistic.intercept_, logistic.n_iter_

In [ ]:
# Plot logistic sigmoid fit
x = np.linspace(0, 1, 200)
y = logistic.predict_proba(x[:, np.newaxis])[:, 1]

fig, ax = plt.subplots(figsize=(6.4, 4.8))
ax.plot([0, 1], [0, 1], "k:", label="Perfect calibration")
ax.plot(x, y, label="Logistic sigmoid fit")

In [ ]:
fig, ax = plot_calibration_curve(
    targets=median_ensemble_val["y"],
    model_probs=[ensemble_probs_logistic_val, *individual_probs_logistic_val],
    labels=["Ensemble"],#, *["Constituent model"] * 5],
    n_bins=20,
)
plot_histogram(arrays=[ensemble_probs_logistic_val], labels=["Ensemble"])

In [ ]:
fig, ax = plot_calibration_curve(
    targets=median_ensemble_test["y"],
    # model_probs=[ensemble_probs_logistic, *individual_probs_logistic],
    model_probs=[ensemble_probs_logistic, *individual_probs_logistic],
    labels=["Ensemble"], #, *["Constituent model"] * 5],
    n_bins=20,
    strategy="uniform",
    # n_bins=1000,
    # strategy="quantile",
)
fig.savefig("calibration_curve_ensemble_logistic.pdf", bbox_inches="tight")

## Isotonic

In [ ]:
isotonic = IsotonicRegression(y_min=0, y_max=1, increasing=True, out_of_bounds="clip")
isotonic.fit(median_ensemble_val["ensemble_probs"].to_numpy(), median_ensemble_val["y"].to_numpy())
ensemble_probs_isotonic = isotonic.transform(median_ensemble_test["ensemble_probs"].to_numpy())
ensemble_probs_isotonic_val = isotonic.transform(median_ensemble_val["ensemble_probs"].to_numpy())

individual_probs_isotonic = []
individual_probs_isotonic_val = []
for i in range(1, 6):
    isotonic_i = IsotonicRegression(y_min=0, y_max=1, increasing=True, out_of_bounds="clip")
    isotonic_i.fit(median_ensemble_val[f"probs {i}"].to_numpy(), median_ensemble_val["y"].to_numpy())
    individual_probs_isotonic += [isotonic_i.transform(median_ensemble_test[f"probs {i}"].to_numpy())]
    individual_probs_isotonic_val += [isotonic_i.transform(median_ensemble_val[f"probs {i}"].to_numpy())]

In [ ]:
isotonic.X_thresholds_[-10:], isotonic.y_thresholds_[-10:]

In [ ]:
median_ensemble_test["ensemble_probs"].sort_values().to_numpy()[-30:]

In [ ]:
plot_histogram(arrays=[ensemble_probs_isotonic, ensemble_probs_isotonic_val], labels=["Ensemble test", "Ensemble validation"])

In [ ]:
x = np.linspace(0, 1, 200)
y = isotonic.transform(x)

fig, ax = plt.subplots(figsize=(6.4, 4.8))
ax.plot([0, 1], [0, 1], "k:", label="Perfect calibration")
# ax.plot(isotonic.X_thresholds_, isotonic.y_thresholds_, "-", marker="o", markersize=3, label="Isotonic")
ax.plot(x, y, label="Isotonic fit")

val_pos = median_ensemble_val["ensemble_probs"][median_ensemble_val["y"] == 1]
val_neg = median_ensemble_val["ensemble_probs"][median_ensemble_val["y"] == 0]
plot_histogram(arrays=[val_neg, val_pos], labels=["Ensemble probs (val-neg)", "Ensemble probs (val-pos)"])

In [ ]:
fig, ax = plot_calibration_curve(
    targets=median_ensemble_val["y"],
    model_probs=[ensemble_probs_isotonic_val, *individual_probs_isotonic_val],
    labels=["Ensemble"],#, *["Constituent model"] * 5],
    n_bins=10,
    strategy="uniform",
    # n_bins=1000,
    # strategy="quantile",
)
plot_histogram(arrays=[ensemble_probs_isotonic_val], labels=["Ensemble"])

In [ ]:
median_ensemble_test["y"].sum()

In [ ]:
ensemble_probs_isotonic

In [ ]:
fig, ax = plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=[ensemble_probs_isotonic, *individual_probs_isotonic],
    labels=["Ensemble"],#, *["Constituent model"] * 5],
    # n_bins=10,
    # strategy="uniform",
    n_bins=1000,
    strategy="quantile",
)
plot_histogram(arrays=[ensemble_probs_isotonic], labels=["Ensemble"])

## MLP

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(8, 8), activation="tanh", solver="adam", max_iter=3000, random_state=0)
mlp.out_activation_ = "sigmoid"
mlp.fit(median_ensemble_val["ensemble_probs"].to_numpy()[:, np.newaxis], median_ensemble_val["y"].to_numpy()[:, np.newaxis])
ensemble_probs_mlp = mlp.predict(median_ensemble_test["ensemble_probs"].to_numpy()[:, np.newaxis]).clip(0)
ensemble_probs_mlp_val = mlp.predict(median_ensemble_val["ensemble_probs"].to_numpy()[:, np.newaxis]).clip(0)

individual_probs_mlp = []
individual_probs_mlp_val = []
for i in range(1, 6):
    mlp_i = MLPRegressor(hidden_layer_sizes=(8, 8), activation="tanh", solver="adam", max_iter=3000, random_state=0)
    mlp_i.out_activation_ = "sigmoid"
    mlp_i.fit(median_ensemble_val[f"probs {i}"].to_numpy()[:, np.newaxis], median_ensemble_val["y"].to_numpy()[:, np.newaxis])
    individual_probs_mlp += [mlp_i.predict(median_ensemble_test[f"probs {i}"].to_numpy()[:, np.newaxis]).clip(0)]
    individual_probs_mlp_val += [mlp_i.predict(median_ensemble_val[f"probs {i}"].to_numpy()[:, np.newaxis]).clip(0)]

In [ ]:
plot_histogram(arrays=[ensemble_probs_mlp, ensemble_probs_mlp_val], labels=["Ensemble test", "Ensemble validation"])

In [ ]:
fig, ax = plot_calibration_curve(
    targets=median_ensemble_val["y"],
    model_probs=[ensemble_probs_mlp_val, *individual_probs_mlp_val],
    labels=["Ensemble"],#, *["Constituent model"] * 5],
    n_bins=15,
    strategy="uniform",
    # n_bins=1000,
    # strategy="quantile",
)
plot_histogram(arrays=[ensemble_probs_mlp_val], labels=["Ensemble"])

In [ ]:
fig, ax = plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=[ensemble_probs_mlp, *individual_probs_mlp],
    labels=["Ensemble"],#, *["Constituent model"] * 5],
    # n_bins=10,
    # strategy="uniform",
    n_bins=1000,
    strategy="quantile",
)
plot_histogram(arrays=[ensemble_probs_mlp], labels=["Ensemble"])

## Plot-making

In [ ]:
def get_pow2_bin_edges(num_bins: int, min_val: float = 0.0, max_val: float = 1.0):
    bin_edges = [i**2 for i in range(1, num_bins, 1)] #np.linspace(0, 1, num_bins + 1)
    bin_edges = np.array(bin_edges) / (num_bins + 1) ** 2
    bin_edges = np.concatenate([[0.0], bin_edges, [1.0]])
    bin_edges = bin_edges * (max_val - min_val) + min_val
    return bin_edges

In [ ]:
get_pow2_bin_edges(10), get_pow2_bin_edges(10).shape

In [ ]:
# All calibration fits

# Logistic sigmoid
x_logistic = np.linspace(0, 1, 200)
y_logistic = logistic.predict_proba(x_logistic[:, np.newaxis])[:, 1]

# Isothonic regression
x_isotonic = np.linspace(0, 1, 200)
y_isotonic = isotonic.transform(x_isotonic)

fig, ax = plt.subplots(figsize=(6.4, 4.8))
ax.plot([0, 1], [0, 1], label="Uncalibrated")
ax.plot(x_logistic, y_logistic, label="Logistic fit")
ax.plot(x_isotonic, y_isotonic, "-", label="Isotonic fit")
ax.set_xlabel("Predicted probability")
ax.set_ylabel("Calibrated probability")
ax.legend()
fig.savefig("calibration_fits_ensemble.pdf", bbox_inches="tight")

In [ ]:
# All calibration curves
arrays = [median_ensemble_test["ensemble_probs"], ensemble_probs_logistic, ensemble_probs_isotonic]#, ensemble_probs_mlp]
labels = ["Ensemble uncalibrated", "Ensemble logistic calibration", "Ensemble isotonic calibration", "Ensemble MLP calibration"]

fig, ax = plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=arrays[0:1],
    labels=labels[0:1], #, *["Constituent model"] * 5],
    n_bins=20,
    strategy="uniform",
)

plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=arrays[1:],
    labels=labels[1:], #, *["Constituent model"] * 5],
    n_bins=1000,
    strategy="quantile",
    ax=ax,
)
fig.savefig("calibration_curves_ensemble.pdf", bbox_inches="tight")
plot_histogram(arrays=arrays, labels=labels)

In [ ]:
# All calibration curves
arrays = [median_ensemble_test["ensemble_probs"], ensemble_probs_logistic, ensemble_probs_isotonic]#, ensemble_probs_mlp]
labels = ["Ensemble uncalibrated", "Ensemble logistic calibration", "Ensemble isotonic calibration", "Ensemble MLP calibration"]

fig, ax = plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=arrays[0:1],
    labels=labels[0:1], #, *["Constituent model"] * 5],
    n_bins=20,
    strategy="uniform",
)

plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=arrays[1:],
    labels=labels[1:], #, *["Constituent model"] * 5],
    n_bins=get_pow2_bin_edges(7),
    ax=ax,
)
# fig.savefig("calibration_curves_ensemble.pdf", bbox_inches="tight")
plot_histogram(arrays=arrays, labels=labels)

In [ ]:
# Brier scores
brier_scores = {
    "uncalibrated": sklearn.metrics.brier_score_loss(median_ensemble_test["y"], median_ensemble_test["ensemble_probs"]),
    "logistic": sklearn.metrics.brier_score_loss(median_ensemble_test["y"], ensemble_probs_logistic),
    "isotonic": sklearn.metrics.brier_score_loss(median_ensemble_test["y"], ensemble_probs_isotonic),
    # "mlp": sklearn.metrics.brier_score_loss(median_ensemble_test["y"], ensemble_probs_mlp),
    "average": sklearn.metrics.brier_score_loss(median_ensemble_test["y"], median_ensemble_test["y"].mean()[np.newaxis].repeat(len(median_ensemble_test))),
}
print("Brier scores: ", brier_scores)

# Brier skill scores
# - uncalibrated reference
brier_skill_scores_uncalibrated = {
    "logistic": 1 - brier_scores["logistic"] / brier_scores["uncalibrated"],
    "isotonic": 1 - brier_scores["isotonic"] / brier_scores["uncalibrated"],
    # "mlp": 1 - brier_scores["mlp"] / brier_scores["uncalibrated"],
}
print("BSS uncalibrated reference: ", brier_skill_scores_uncalibrated)

# - average target reference
brier_skill_scores_mean = {
    "logistic": 1 - brier_scores["logistic"] / brier_scores["average"],
    "isotonic": 1 - brier_scores["isotonic"] / brier_scores["average"],
    # "mlp": 1 - brier_scores["mlp"] / brier_scores["average"],
}
print("BSS mean reference: ", brier_skill_scores_mean)


In [ ]:
# F1-score of model and call-taker as function of model output probabilities

def plot_f1_score_vs_probabilities(targets, model_probs, model_preds, human_preds, model_label: str, target_label: str = "Call-taker", num_bins: int = 10):

    bin_edges = get_pow2_bin_edges(num_bins)
    hist, bin_edges = np.histogram(model_probs, bins=bin_edges)
    print("Bin edges: ", bin_edges)

    bin_widths = bin_edges[1:] - bin_edges[:-1]

    fig1 = plt.figure(figsize=(6.4, 4.8))
    ax1 = fig1.gca()
    ax1.bar(bin_edges[:-1], hist, width=bin_widths, align="edge", alpha=0.5, label=model_label)
    ax1.set_yscale("log")
    ax1.set_xlabel("Predicted probability")
    ax1.set_ylabel("Count")

    f1s_model = []
    f1s_calltaker = []
    for low, high in zip(bin_edges[:-1], bin_edges[1:]):
        cond = (model_probs >= low) & (model_probs < high)
        
        f1_score = sklearn.metrics.f1_score(targets[cond], model_preds[cond])
        f1s_model.append(f1_score)
        
        f1_score_calltaker = sklearn.metrics.f1_score(targets[cond], human_preds[cond])
        f1s_calltaker.append(f1_score_calltaker)

    f1s_model = np.array(f1s_model)
    f1s_calltaker = np.array(f1s_calltaker)

    fig2 = plt.figure(figsize=(6.4, 4.8))
    ax2 = fig2.gca()

    ax2.bar(bin_edges[:-1], f1s_model, width=bin_widths, align="edge", alpha=0.5, label=model_label)
    ax2.bar(bin_edges[:-1], f1s_calltaker, width=bin_widths, align="edge", alpha=0.5, label=target_label)

    ax2.set_ylabel("F1-score")

    ax2.set_xlabel("Predicted probability")
    ax2.legend(loc="upper left")
    return (fig1, ax1), (fig2, ax2)

In [ ]:
plot_f1_score_vs_probabilities(
    median_ensemble_test["y"],
    median_ensemble_test["ensemble_probs"],
    median_ensemble_test["ensemble_preds"],
    median_ensemble_test["h"],
    "Uncalibrated ensemble",
    "Call-taker",
    num_bins=10
)

In [ ]:
(fig1, ax1), (fig2, ax2) = plot_f1_score_vs_probabilities(
    median_ensemble_test["y"],
    ensemble_probs_isotonic,
    median_ensemble_test["ensemble_preds"],
    median_ensemble_test["h"],
    "Calibrated ensemble (isotonic)",
    "Call-taker",
    num_bins=8
)
# ax1.set_xlim(0, 0.64)
# ax2.set_xlim(0, 0.64)

fig1.savefig("predicted_probability_histogram.pdf", bbox_inches="tight")
fig2.savefig("f1_score_vs_predicted_probability_ensemble_calltaker.pdf", bbox_inches="tight")

In [ ]:
plot_f1_score_vs_probabilities(
    median_ensemble_val["y"],
    ensemble_probs_isotonic_val,
    median_ensemble_val["ensemble_preds"],
    median_ensemble_val["h"],
    "Calibrated ensemble (isotonic) [val]",
    "Call-taker [val]",
    num_bins=10
)

## Plot-making with bootstrap CIs

In [ ]:
# Median ensemble
human_predictions_test = median_ensemble_test["h"].to_numpy()
ensemble_predictions_test = median_ensemble_test["ensemble_preds"].to_numpy()
uncalibrated_probabilities_test = median_ensemble_test[f"ensemble_probs"].to_numpy()
targets_test = median_ensemble_test["y"].to_numpy().astype(int)

uncalibrated_probabilities_val = median_ensemble_val[f"ensemble_probs"].to_numpy()
targets_val = median_ensemble_val["y"].to_numpy().astype(int)

# All ensembles
# uncalibrated_probabilities_test = [all_ensembles_test[f"ensemble {i} probs"] for i in range(1, 12)]
# targets_test = [all_ensembles_test["y"]] * 11

# uncalibrated_probabilities_val = [all_ensembles_val[f"ensemble {i} probs"] for i in range(1, 12)]
# targets_val = [all_ensembles_val["y"]] * 11

In [ ]:
# Number of bootstrap iterations
n_bootstrap = 3000

uncalibrated_probabilities_val_resampled = []
targets_val_sampled = []

human_preds_resampled = []
model_preds_resampled = []
uncalibrated_probabilities_test_resampled = []
isotonic_probabilities_test_resampled = []
logistic_probabilities_test_resampled = []
targets_test_resampled = []

isotonic_probabilities_test = isotonic.transform(uncalibrated_probabilities_test)
logistic_probabilities_test = logistic.predict_proba(uncalibrated_probabilities_test[:, np.newaxis])[:, 1]

bin_edges = get_pow2_bin_edges(10)

for _ in tqdm(range(n_bootstrap)):
    # Resample the data with replacement
    arrays = [targets_test, ensemble_predictions_test, human_predictions_test, uncalibrated_probabilities_test, isotonic_probabilities_test, logistic_probabilities_test]
    arrays_resampled = sklearn.utils.resample(*arrays, replace=True)
    
    targets_test_resampled.append(arrays_resampled[0])
    model_preds_resampled.append(arrays_resampled[1])
    human_preds_resampled.append(arrays_resampled[2])
    uncalibrated_probabilities_test_resampled.append(arrays_resampled[3])
    isotonic_probabilities_test_resampled.append(arrays_resampled[4])
    logistic_probabilities_test_resampled.append(arrays_resampled[5])
    


In [ ]:
num_bins = 7

uncalibrated_calibration_curves = []
isotonic_calibration_curves = []
logistic_calibration_curves = []

for i in tqdm(range(n_bootstrap)):
    uncalibrated_calibration_curve_i = calibration_curve(
        targets_test_resampled[i],
        uncalibrated_probabilities_test_resampled[i],
        n_bins=num_bins,
        strategy="uniform",
    )
    uncalibrated_calibration_curves.append(uncalibrated_calibration_curve_i)
    
    isotonic_calibration_curve_i = calibration_curve(
        targets_test_resampled[i],
        isotonic_probabilities_test_resampled[i],
        n_bins=get_pow2_bin_edges(num_bins),
    )
    isotonic_calibration_curves.append(isotonic_calibration_curve_i)
    
    logistic_calibration_curve_i = calibration_curve(
        targets_test_resampled[i],
        logistic_probabilities_test_resampled[i],
        n_bins=get_pow2_bin_edges(num_bins),
    )
    logistic_calibration_curves.append(logistic_calibration_curve_i)


uncalibrated_calibration_curve = calibration_curve(
    targets_test,
    uncalibrated_probabilities_test,
    n_bins=num_bins,
    strategy="uniform",
)

isotonic_calibration_curve = calibration_curve(
    targets_test,
    isotonic_probabilities_test,
    n_bins=get_pow2_bin_edges(num_bins),
)

logistic_calibration_curve = calibration_curve(
    targets_test,
    logistic_probabilities_test,
    n_bins=get_pow2_bin_edges(num_bins),
)

In [ ]:
print(all([c[0].shape == logistic_calibration_curves[0][0].shape for c in logistic_calibration_curves]))
print(all([c[0].shape == isotonic_calibration_curves[0][0].shape for c in isotonic_calibration_curves]))
print(all([c[0].shape == uncalibrated_calibration_curves[0][0].shape for c in uncalibrated_calibration_curves]))

In [ ]:
# Calculate the percentiles for the confidence intervals
def get_calibration_confidence_intervals(calibration_curve: np.ndarray, bootstrap_calibration_curves: List[np.ndarray], alpha: float = 0.05):
    lower_bound = np.nanpercentile(bootstrap_calibration_curves, alpha / 2, axis=0)
    upper_bound = np.nanpercentile(bootstrap_calibration_curves, 100 - alpha / 2, axis=0)
    xerr = np.abs(np.stack([lower_bound[1], upper_bound[1]]) - calibration_curve[1])
    yerr = np.abs(np.stack([lower_bound[0], upper_bound[0]]) - calibration_curve[0])
    return xerr, yerr


uncalibrated_xerr, uncalibrated_yerr = get_calibration_confidence_intervals(uncalibrated_calibration_curve, uncalibrated_calibration_curves, alpha=0.05)
isotonic_xerr, isotonic_yerr = get_calibration_confidence_intervals(isotonic_calibration_curve, isotonic_calibration_curves, alpha=0.05)
logistic_xerr, logistic_yerr = get_calibration_confidence_intervals(logistic_calibration_curve, logistic_calibration_curves, alpha=0.05)

In [ ]:
isotonic_yerr

In [ ]:
fig, ax = plt.subplots(figsize=(6.4, 4.8))
ax.plot([0, 1], [0, 1], "k:", label="Perfect calibration")

# ax.plot(uncalibrated_calibration_curve[1], uncalibrated_calibration_curve[0], marker="o", markersize=3, label="Ensemble uncalibrated")
ax.errorbar(
    x=uncalibrated_calibration_curve[1],
    y=uncalibrated_calibration_curve[0],
    xerr=uncalibrated_xerr,
    yerr=uncalibrated_yerr,
    marker="o",
    markersize=3,
    capsize=2,
    label="Ensemble uncalibrated"
)

ax.errorbar(
    x=logistic_calibration_curve[1],
    y=logistic_calibration_curve[0],
    xerr=logistic_xerr,
    yerr=logistic_yerr,
    marker="o",
    markersize=3,
    capsize=2,
    label="Ensemble logistic calibration"
)

ax.errorbar(
    x=isotonic_calibration_curve[1],
    y=isotonic_calibration_curve[0],
    xerr=isotonic_xerr,
    yerr=isotonic_yerr,
    marker="o",
    markersize=3,
    capsize=2,
    label="Ensemble isotonic calibration"
)
ax.set_xlabel("Mean predicted probability")
ax.set_ylabel("Fraction of positives")
ax.legend()

fig.savefig("calibration_curves_ensemble_with_cis.pdf", bbox_inches="tight")

In [ ]:
targets = median_ensemble_test["y"]
human_preds = median_ensemble_test["h"]
model_preds = median_ensemble_test["ensemble_preds"]
model_probs = isotonic_probabilities_test

targets_resampled = targets_test_resampled
human_preds_resampled = human_preds_resampled
model_preds_resampled = model_preds_resampled
model_probs_resampled = isotonic_probabilities_test_resampled

num_bins = 6
bin_edges = get_pow2_bin_edges(num_bins)
hist, bin_edges = np.histogram(model_probs, bins=bin_edges)
print("Bin edges: ", bin_edges)

bin_widths = bin_edges[1:] - bin_edges[:-1]

# fig1 = plt.figure(figsize=(6.4, 4.8))
# ax1 = fig1.gca()
# ax1.bar(bin_edges[:-1], hist, width=bin_widths, align="edge", alpha=0.5, label=model_label)
# ax1.set_yscale("log")
# ax1.set_xlabel("Predicted probability")
# ax1.set_ylabel("Count")

In [ ]:
def get_bootstrap_confidence_intervals_for_scalar(main_estimate: np.ndarray, bootstrap_estimates: List[np.ndarray], alpha: float = 0.05):
    lower_bound = np.nanpercentile(bootstrap_estimates, alpha / 2, axis=0)
    upper_bound = np.nanpercentile(bootstrap_estimates, 100 - alpha / 2, axis=0)
    yerr = np.abs(np.stack([lower_bound, upper_bound]) - main_estimate)
    return yerr


precision_model = []
precision_calltaker = []
recall_model = []
recall_calltaker = []
f1s_model = []
f1s_calltaker = []
for low, high in zip(bin_edges[:-1], bin_edges[1:]):
    cond = (model_probs >= low) & (model_probs < high)
    
    metrics_model = sklearn.metrics.precision_recall_fscore_support(targets[cond], model_preds[cond], zero_division=np.nan, average="binary", pos_label=1)
    metrics_calltaker = sklearn.metrics.precision_recall_fscore_support(targets[cond], human_preds[cond], zero_division=np.nan, average="binary", pos_label=1)
    
    precision_model.append(metrics_model[0])
    precision_calltaker.append(metrics_calltaker[0])
    
    recall_model.append(metrics_model[1])
    recall_calltaker.append(metrics_calltaker[1])
    
    f1s_model.append(metrics_model[2])
    f1s_calltaker.append(metrics_calltaker[2])

precision_model = np.array(precision_model)
precision_calltaker = np.array(precision_calltaker)

recall_model = np.array(recall_model)
recall_calltaker = np.array(recall_calltaker)

f1s_model = np.array(f1s_model)
f1s_calltaker = np.array(f1s_calltaker)


bootstrap_precision_model = []
bootstrap_precision_calltaker = []
bootstrap_recall_model = []
bootstrap_recall_calltaker = []
bootstrap_f1s_model = []
bootstrap_f1s_calltaker = []
for i in tqdm(range(n_bootstrap), desc="Bootstrap iterations"):
    precision_model_i = []
    precision_calltaker_i = []
    recall_model_i = []
    recall_calltaker_i = []
    f1s_model_i = []
    f1s_calltaker_i = []
    for low, high in zip(bin_edges[:-1], bin_edges[1:]):
        cond = (model_probs_resampled[i] >= low) & (model_probs_resampled[i] < high)
        
        metrics_model = sklearn.metrics.precision_recall_fscore_support(targets_resampled[i][cond], model_preds_resampled[i][cond], zero_division=np.nan, average="binary", pos_label=1)
        metrics_calltaker = sklearn.metrics.precision_recall_fscore_support(targets_resampled[i][cond], human_preds_resampled[i][cond], zero_division=np.nan, average="binary", pos_label=1)
        
        precision_model_i.append(metrics_model[0])
        precision_calltaker_i.append(metrics_calltaker[0])
        
        recall_model_i.append(metrics_model[1])
        recall_calltaker_i.append(metrics_calltaker[1])
        
        f1s_model_i.append(metrics_model[2])
        f1s_calltaker_i.append(metrics_calltaker[2])

    bootstrap_precision_model.append(precision_model_i)
    bootstrap_precision_calltaker.append(precision_calltaker_i)

    bootstrap_recall_model.append(recall_model_i)
    bootstrap_recall_calltaker.append(recall_calltaker_i)

    bootstrap_f1s_model.append(f1s_model_i)
    bootstrap_f1s_calltaker.append(f1s_calltaker_i)

bootstrap_precision_model = np.array(bootstrap_precision_model)
bootstrap_precision_calltaker = np.array(bootstrap_precision_calltaker)
bootstrap_recall_model = np.array(bootstrap_recall_model)
bootstrap_recall_calltaker = np.array(bootstrap_recall_calltaker)
bootstrap_f1s_model = np.array(bootstrap_f1s_model)
bootstrap_f1s_calltaker = np.array(bootstrap_f1s_calltaker)

In [ ]:
metrics_calltaker

In [ ]:
precision_model_yerr = get_bootstrap_confidence_intervals_for_scalar(precision_model, bootstrap_precision_model, alpha=0.05)
precision_calltaker_yerr = get_bootstrap_confidence_intervals_for_scalar(precision_calltaker, bootstrap_precision_calltaker, alpha=0.05)

recall_model_yerr = get_bootstrap_confidence_intervals_for_scalar(recall_model, bootstrap_recall_model, alpha=0.05)
recall_calltaker_yerr = get_bootstrap_confidence_intervals_for_scalar(recall_calltaker, bootstrap_recall_calltaker, alpha=0.05)

f1s_model_yerr = get_bootstrap_confidence_intervals_for_scalar(f1s_model, bootstrap_f1s_model, alpha=0.05)
f1s_calltaker_yerr = get_bootstrap_confidence_intervals_for_scalar(f1s_calltaker, bootstrap_f1s_calltaker, alpha=0.05)

In [ ]:
fig2 = plt.figure(figsize=(6.4, 4.8))
ax2 = fig2.gca()

ax2.errorbar(
    x=bin_edges[:-1],
    y=f1s_model,
    yerr=f1s_model_yerr,
    marker="o",
    markersize=3,
    capsize=2,
    label="Calibrated ensemble (isotonic)"
)

ax2.errorbar(
    x=bin_edges[:-1],
    y=f1s_calltaker,
    yerr=f1s_calltaker_yerr,
    marker="o",
    markersize=3,
    capsize=2,
    label="Call-taker"
)

# ax2.bar(bin_edges[:-1], f1s_model, width=bin_widths, align="edge", alpha=0.5, label="Calibrated ensemble (isotonic)")
# ax2.bar(bin_edges[:-1], f1s_calltaker, width=bin_widths, align="edge", alpha=0.5, label="Call-taker")

ax2.set_ylabel("F1-score")

ax2.set_xlabel("Predicted probability")
ax2.legend(loc="upper left")
fig2.savefig("f1_score_vs_predicted_probability_ensemble_calltaker_with_cis.pdf", bbox_inches="tight")

In [ ]:
fig2 = plt.figure(figsize=(6.4, 4.8))
ax2 = fig2.gca()

ax2.errorbar(
    x=bin_edges[:-1],
    y=precision_model,
    yerr=precision_model_yerr,
    marker="o",
    markersize=3,
    capsize=2,
    label="Calibrated ensemble (isotonic)"
)

ax2.errorbar(
    x=bin_edges[:-1],
    y=precision_calltaker,
    yerr=precision_calltaker_yerr,
    marker="o",
    markersize=3,
    capsize=2,
    label="Call-taker"
)

# ax2.bar(bin_edges[:-1], precision_model, width=bin_widths, align="edge", alpha=0.5, label="Calibrated ensemble (isotonic)")
# ax2.bar(bin_edges[:-1], precision_calltaker, width=bin_widths, align="edge", alpha=0.5, label="Call-taker")

ax2.set_ylabel("Precision")
ax2.set_xlabel("Predicted probability")
ax2.legend(loc="upper left")
fig2.savefig("precision_vs_predicted_probability_ensemble_calltaker_with_cis.pdf", bbox_inches="tight")

In [ ]:
fig2 = plt.figure(figsize=(6.4, 4.8))
ax2 = fig2.gca()

ax2.errorbar(
    x=bin_edges[:-1],
    y=recall_model,
    yerr=recall_model_yerr,
    marker="o",
    markersize=3,
    capsize=2,
    label="Calibrated ensemble (isotonic)"
)

ax2.errorbar(
    x=bin_edges[:-1],
    y=recall_calltaker,
    yerr=recall_calltaker_yerr,
    marker="o",
    markersize=3,
    capsize=2,
    label="Call-taker"
)

# ax2.bar(bin_edges[:-1], recall_model, width=bin_widths, align="edge", alpha=0.5, label="Calibrated ensemble (isotonic)")
# ax2.bar(bin_edges[:-1], recall_calltaker, width=bin_widths, align="edge", alpha=0.5, label="Call-taker")

ax2.set_ylabel("Recall")
ax2.set_xlabel("Predicted probability")
ax2.legend(loc="upper left")
fig2.savefig("recall_vs_predicted_probability_ensemble_calltaker_with_cis.pdf", bbox_inches="tight")

In [ ]:
num_main_positives = []
num_main_negatives = []
for low, high in zip(bin_edges[:-1], bin_edges[1:]):
    cond = (model_probs_resampled[i] >= low) & (model_probs_resampled[i] < high)
    
    num_main_positives += [np.sum(targets[cond])]
    num_main_negatives += [np.sum(1 - targets[cond])]
    
    metrics_model = sklearn.metrics.precision_recall_fscore_support(targets[cond], model_preds[cond], zero_division=np.nan, average="binary", pos_label=1)

num_main_positives = np.array(num_main_positives)
num_main_negatives = np.array(num_main_negatives)

num_positives = []
num_negatives = []
for i in tqdm(range(n_bootstrap), desc="Bootstrap iterations"):
    num_positives_i = []
    num_negatives_i = []
    for low, high in zip(bin_edges[:-1], bin_edges[1:]):
        cond = (model_probs_resampled[i] >= low) & (model_probs_resampled[i] < high)
        
        num_positives_i += [np.sum(targets_resampled[i][cond])]
        num_negatives_i += [np.sum(1 - targets_resampled[i][cond])]
        
        metrics_model = sklearn.metrics.precision_recall_fscore_support(targets_resampled[i][cond], model_preds_resampled[i][cond], zero_division=np.nan, average="binary", pos_label=1)

    num_positives.append(num_positives_i)
    num_negatives.append(num_negatives_i)
    
num_positives = np.array(num_positives)
num_negatives = np.array(num_negatives)



In [ ]:
pos_err = get_bootstrap_confidence_intervals_for_scalar(num_main_positives, num_positives, alpha=0.05)
neg_err = get_bootstrap_confidence_intervals_for_scalar(num_main_negatives, num_negatives, alpha=0.05)

In [ ]:
num_main_positives, num_main_negatives

In [ ]:
# Compute Brier scores with CIs
brier_scores = []
for i in tqdm(range(n_bootstrap), desc="Bootstrap iterations"):
    brier_score = sklearn.metrics.brier_score_loss(targets_resampled[i], model_probs_resampled[i])
    brier_scores.append(brier_score)
brier_scores = np.array(brier_scores)

brier_score = sklearn.metrics.brier_score_loss(targets, model_probs)

brier_score_yerr = get_bootstrap_confidence_intervals_for_scalar(brier_score, brier_scores, alpha=0.05)

# Compute Brier skill scores with CIs
brier_skill_scores = []
for i in tqdm(range(n_bootstrap), desc="Bootstrap iterations"):
    mean = np.array([targets_resampled[i].mean()] * len(targets_resampled[i]))
    brier_skill_score = 1 - sklearn.metrics.brier_score_loss(targets_resampled[i], model_probs_resampled[i]) / sklearn.metrics.brier_score_loss(targets_resampled[i], mean)
    brier_skill_scores.append(brier_skill_score)
brier_skill_scores = np.array(brier_skill_scores)

mean = np.array([targets.mean()] * len(targets))
brier_skill_score = 1 - sklearn.metrics.brier_score_loss(targets, model_probs) / sklearn.metrics.brier_score_loss(targets, mean)

brier_skill_score_yerr = get_bootstrap_confidence_intervals_for_scalar(brier_skill_score, brier_skill_scores, alpha=0.05)

brier_score_yerr[0] = - brier_score_yerr[0]
brier_skill_score_yerr[0] = - brier_skill_score_yerr[0]
print("Brier score: ", brier_score, brier_score_yerr, brier_score_yerr + brier_score)
print("Brier skill score: ", brier_skill_score, brier_skill_score_yerr, brier_skill_score_yerr + brier_skill_score)

In [ ]:
all_ensembles_test

In [ ]:
raise Exception("Stop here")

# Ensemble of ensembles

In [ ]:
all_ensembles_test

In [ ]:
all_ensembles_val

In [ ]:
all_ensembles_test_probs = [all_ensembles_test[f"ensemble {i} probs"] for i in range(1, 12)]
labels = [f"Ensemble {i}" for i in range(1, 12)]

plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=all_ensembles_test_probs,
    labels=labels,
    n_bins=20,
)

In [ ]:
all_ensembles_test_probs = np.stack([all_ensembles_test[f"ensemble {i} probs"] for i in range(1, 12)])
all_ensembles_test_preds = np.stack([all_ensembles_test[f"ensemble {i} preds"] for i in range(1, 12)])
all_ensembles_val_probs = np.stack([all_ensembles_val[f"ensemble {i} probs"] for i in range(1, 12)])
all_ensembles_val_preds = np.stack([all_ensembles_val[f"ensemble {i} preds"] for i in range(1, 12)])
all_ensembles_test_preds.shape

In [ ]:
med_ensemble_test = median_ensemble_test["ensemble_probs"]
# majority_vote_test = np.mean(all_ensembles_test_preds, axis=0) > 0.5
super_ensemble_probs_test = scipy.stats.hmean(all_ensembles_test_probs, axis=0)
mean_ensemble_probs_test = np.mean(all_ensembles_test_probs, axis=0)

med_ensemble_val = median_ensemble_val["ensemble_probs"]
# majority_vote_val = np.mean(all_ensembles_test_preds, axis=0) > 0.5
super_ensemble_probs_val = scipy.stats.hmean(all_ensembles_val_probs, axis=0)
mean_ensemble_probs_val = np.mean(all_ensembles_val_probs, axis=0)

In [ ]:
plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=[med_ensemble_test, super_ensemble_probs_test, mean_ensemble_probs_test],
    labels=["Median ensemble", "Harmonic mean of all ensembles", "Mean of all ensembles"],
    n_bins=20,
)
plot_histogram(arrays=[med_ensemble_test, super_ensemble_probs_test, mean_ensemble_probs_test], labels=["Majority vote", "Super ensemble", "Mean ensemble"])

## Platt scaling

In [ ]:
logistic = LogisticRegression(penalty="none", fit_intercept=True)
logistic.fit(median_ensemble_val["ensemble_probs"].to_numpy()[:, np.newaxis], median_ensemble_val["y"].to_numpy())
ensemble_probs_logistic = logistic.predict_proba(median_ensemble_test["ensemble_probs"].to_numpy()[:, np.newaxis])[:, 1]
ensemble_probs_logistic_val = logistic.predict_proba(median_ensemble_val["ensemble_probs"].to_numpy()[:, np.newaxis])[:, 1]


logistic = LogisticRegression(penalty="none", fit_intercept=True)
logistic.fit(super_ensemble_probs_val[:, np.newaxis], median_ensemble_val["y"].to_numpy())
ensemble_probs_logistic = logistic.predict_proba(super_ensemble_probs_test[:, np.newaxis])[:, 1]

In [ ]:
logistic.coef_, logistic.intercept_, logistic.n_iter_

In [ ]:
ensemble_probs_logistic.min(), ensemble_probs_logistic.max()

In [ ]:
plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=[ensemble_probs_logistic],
    labels=["Ensemble"],
    n_bins=20,
)
plot_histogram(arrays=[ensemble_probs_logistic], labels=["Ensemble"])

In [ ]:
is_old = median_ensemble_test["age"] >= 65
plot_calibration_curve(
    targets=[median_ensemble_test["y"][is_old], median_ensemble_test["y"][~is_old]],
    model_probs=[ensemble_probs_logistic[is_old], ensemble_probs_logistic[~is_old]],
    labels=["65+", "18-65"],
    n_bins=20,
)
plot_histogram(
    arrays=[ensemble_probs_logistic[is_old], ensemble_probs_logistic[~is_old]],
    labels=["65+", "18-65"],
)

## Isotonic

In [ ]:
isotonic = IsotonicRegression(y_min=0, y_max=1, increasing=True, out_of_bounds="clip")
isotonic.fit(super_ensemble_probs_val, median_ensemble_val["y"])
ensemble_probs_isotonic = isotonic.transform(super_ensemble_probs_test)

In [ ]:
plot_histogram(arrays=[ensemble_probs_isotonic], labels=["Ensemble validation"])

In [ ]:
fig, ax = plt.subplots(figsize=(6.4, 4.8))
ax.plot([0, 1], [0, 1], "k:", label="Perfect calibration")
ax.plot(isotonic.X_thresholds_, isotonic.y_thresholds_, "-", marker="o", markersize=3, label="Isotonic")

In [ ]:
plot_calibration_curve(
    targets=median_ensemble_test["y"],
    model_probs=[ensemble_probs_isotonic],
    labels=["Ensemble"],
    n_bins=20,
)
plot_histogram(arrays=[ensemble_probs_isotonic], labels=["Ensemble"])